In [1]:
import dolfin as dl
import hippylib as hl
import matplotlib.pyplot as plt
import numpy as np

from spin.core import problem
from spin.fenics import converter as fex_converter
from spin.hippylib import misfit as hlx_misfit
from spin.hippylib import optimization as hlx_optimization
from spin.hippylib import prior as hlx_prior

In [2]:
mesh = dl.IntervalMesh(100, -1, 1)
problem_settings = problem.SPINProblemSettings(
    mesh=mesh,
    pde_type="mean_exit_time",
    inference_type="drift_only",
    drift=(("-x[0]",)),
    log_squared_diffusion=("std::log(1.0)",),
)
problem_builder = problem.SPINProblemBuilder(problem_settings)
spin_problem = problem_builder.build()

In [3]:
forward_vector = spin_problem.hippylib_variational_problem.generate_state()
adjoint_vector = spin_problem.hippylib_variational_problem.generate_state()
parameter_vector = spin_problem.hippylib_variational_problem.generate_parameter()
parameter_vector.set_local(-np.linspace(-1, 1, parameter_vector.size()))
state_list = [forward_vector, parameter_vector, adjoint_vector]
spin_problem.hippylib_variational_problem.solveFwd(forward_vector, state_list)
forward_array = fex_converter.convert_to_numpy(
    forward_vector, spin_problem.function_space_variables
)

In [4]:
prior_settings = hlx_prior.PriorSettings(
    function_space=spin_problem.function_space_parameters,
    mean=("-0.5*x[0]",),
    variance=("0.1",),
    correlation_length=("0.1",),
)
prior_builder = hlx_prior.BilaplacianVectorPriorBuilder(prior_settings)
spin_prior = prior_builder.build()

In [5]:
misfit_settings = hlx_misfit.MisfitSettings(
    function_space=spin_problem.function_space_variables,
    observation_points=spin_problem.coordinates_variables,
    observation_values=forward_array + np.random.normal(0, 0.01, forward_array.shape),
    noise_variance=0.01 * np.ones(forward_array.shape),
)
misfit_builder = hlx_misfit.MisfitBuilder(misfit_settings)
spin_misfit = misfit_builder.build()

In [6]:
inference_model = hl.Model(
    spin_problem.hippylib_variational_problem, spin_prior.hippylib_prior, spin_misfit
)

In [7]:
optimization_settings = hlx_optimization.SolverSettings(
    relative_tolerance=1e-3, absolute_tolerance=1e-3
)
initial_guess = spin_prior.mean_array
newton_solver = hlx_optimization.NewtonCGSolver(optimization_settings, inference_model)
map_estimate = newton_solver.solve(initial_guess)


It  cg_it cost            misfit          reg             (g,dm)          ||g||L2        alpha          tolcg         
  1   1    6.726159e+01    4.764706e+01    1.961452e+01   -1.396474e+03   2.090330e+03   1.000000e+00   5.000000e-01
  2   1    4.123491e+01    6.308298e+00    3.492661e+01   -5.535262e+01   2.476264e+02   1.000000e+00   3.441843e-01
  3   3    4.105982e+01    5.722487e+00    3.533733e+01   -4.058874e-01   7.075869e+00   1.000000e+00   5.818117e-02
